In [7]:
import pandas as pd 
import os 


In [9]:
pwd = os.getcwd()

In [10]:
dataset = pd.read_excel(pwd + "\\Data - Survey Monkey Output.xlsx", sheet_name="Edited_Data")

In [1]:
##print(dataset)

In [11]:
dataset_modified = dataset.copy()
##dataset_modified

In [2]:
##print(dataset_modified.columns)

In [13]:

columns_to_drop = ['Start Date', 'End Date', 'Email Address','First Name', 'Last Name', 'Custom Data 1']
columns_to_drop

['Start Date',
 'End Date',
 'Email Address',
 'First Name',
 'Last Name',
 'Custom Data 1']

In [14]:
dataset_modified = dataset_modified.drop(columns=columns_to_drop)
##dataset_modified

In [15]:
id_var = list(dataset_modified.columns)[:8]
value_var = list(dataset_modified.columns)[8: ]
##value_vars

In [16]:
dataset_melted = dataset_modified.melt(id_vars=id_var, value_vars=value_var,
 var_name="Question + Subquestion", value_name="Answer")
##dataset_melted

In [18]:
Questions_import = pd.read_excel(pwd + "\\Data - Survey Monkey Output.xlsx", 
            sheet_name="Question")
##Questions_import

In [19]:
Questions = Questions_import.copy()
questions_to_drop =["Raw Questions","Raw Subquestions","Subquestions"]
Questions = Questions.drop(columns=questions_to_drop)

In [20]:
##Questions

In [21]:
dataset_merged = pd.merge(left=dataset_melted, right=Questions, how="left",
                left_on="Question + Subquestion", right_on="Question + Subquestion")
##print("Original Data", len(dataset_melted))
##print("Merged Data", len(dataset_merged))
##print(dataset_merged)

In [22]:
respondents = dataset_merged[dataset_merged["Answer"].notna()]
respondents = respondents.groupby("Question")["Respondent ID"].nunique().reset_index()
respondents.rename(columns={"Respondent ID" : "Respondents"}, inplace = True)
##respondents

In [23]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondents, how="left",
                left_on="Question", right_on="Question")
##print("Original Data", len(dataset_merged))
##print("Merged Data", len(dataset_merged_two))
##print(dataset_merged_two)

In [24]:
same_answer = dataset_merged ##[dataset_merged["Answer"].notna()]
same_answer = same_answer.groupby(["Question + Subquestion", "Answer"])["Respondent ID"].nunique().reset_index()
same_answer.rename(columns={"Respondent ID" : "Same_Answer"}, inplace = True)
##same_answer

In [25]:
from xml.dom.pulldom import IGNORABLE_WHITESPACE


dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left",
                left_on=["Question + Subquestion", "Answer"], right_on=["Question + Subquestion", "Answer"])
dataset_merged_three["Same_Answer"].fillna(0, inplace=True)
##print("Original Data", len(dataset_merged))
##print("Merged Data", len(dataset_merged_three))
##print(dataset_merged_three)

In [26]:
output = dataset_merged_three.copy()
output.rename(columns={"Identify which division you work in.-Response": "Division Primary",
"Identify which division you work in.-Other (please specify)" : "Division Secondary",
"Which of the following best describes your position level?-Response":"Position",
"Which generation are you apart of?-Response":"Generation",
"Please select the gender in which you identify.-Response": "Gender",
"Which duration range best aligns with your tenure at your company?-Response": "Tenure",
"Which of the following best describes your employment type?-Response":"Employment Type"}, inplace=True)
##output

In [37]:
output.to_excel(pwd + "\\Final_Output.xlsx", index=False)